# RSL (Relative Stärke Levy) Aktien-Screening-Strategie

## Umfassende Analyse mit detaillierten Finanzkennzahlen

Dieses Notebook implementiert die **Relative Stärke Levy (RSL)** Strategie zur Bewertung und Rangfolge von S&P 500 Aktien mit erweiterten Kennzahlen.

### Was ist RSL?
Der Relative Stärke Levy Indikator wurde 1967 von Robert Levy entwickelt. Er misst das Momentum einer Aktie, indem er den aktuellen Kurs mit dem historischen Durchschnitt vergleicht.

**Formel:**
```
RSL = Aktueller Kurs / SMA(Kurs, N Perioden)
```

### Enthaltene Analysen:
- RSL-Berechnung über mehrere Zeiträume (3M, 6M, 12M)
- Gleitende Durchschnitte (50-Tage, 200-Tage)
- Relative Stärke vs. S&P 500 Index
- Umfassende Finanzkennzahlen (KGV, Dividende, Beta, etc.)
- Professioneller Excel-Bericht mit 5 Blättern

**Geschätzte Ausführungszeit: 10-15 Minuten**

---

## 1. Installation & Imports

In [ ]:
# Erforderliche Pakete installieren
!pip install yfinance pandas openpyxl beautifulsoup4 lxml tqdm xlsxwriter --quiet
print("Alle Pakete erfolgreich installiert!")

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm
import time
import warnings
import os

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Bibliotheken erfolgreich importiert!")
print(f"Ausführungsdatum: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")

## 2. Konfiguration

In [ ]:
# =============================================================================
# KONFIGURATION
# =============================================================================

# RSL-Berechnungsperioden (in Handelstagen)
RSL_PERIODE_3M = 63    # ~3 Monate
RSL_PERIODE_6M = 130   # ~6 Monate (Levys Original: 27 Wochen)
RSL_PERIODE_12M = 252  # ~12 Monate

# Gleitende Durchschnitte
MA_50 = 50
MA_200 = 200

# Datenabruf-Zeitraum
RUECKBLICK_TAGE = 400  # Ausreichend für 200-Tage-MA

# Rate-Limiting
API_VERZOEGERUNG = 0.15

# Top-Performer Schwellenwert
TOP_PROZENT = 0.25

# Ausgabedatei
ZEITSTEMPEL = datetime.now().strftime('%Y-%m-%d_%H-%M')
AUSGABE_DATEI = f"RSL_SP500_Rangliste_{ZEITSTEMPEL}.xlsx"

print("Konfiguration geladen:")
print(f"  - RSL-Perioden: 3M ({RSL_PERIODE_3M}d), 6M ({RSL_PERIODE_6M}d), 12M ({RSL_PERIODE_12M}d)")
print(f"  - Gleitende Durchschnitte: {MA_50}d, {MA_200}d")
print(f"  - Top-Performer: Obere {int(TOP_PROZENT*100)}%")
print(f"  - Ausgabedatei: {AUSGABE_DATEI}")

## 3. Datenabruf-Funktionen

In [ ]:
def hole_sp500_ticker():
    """
    S&P 500 Unternehmensticker und -namen von Wikipedia abrufen.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    print("Lade S&P 500 Ticker von Wikipedia...")
    
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find('table', {'id': 'constituents'})
        
        if table is None:
            tables = soup.find_all('table', {'class': 'wikitable'})
            for t in tables:
                if t.find('th', string=lambda x: x and 'Symbol' in x):
                    table = t
                    break
        
        df = pd.read_html(str(table))[0]
        df.columns = df.columns.str.strip()
        
        result = pd.DataFrame({
            'Symbol': df['Symbol'].str.strip().str.replace('.', '-', regex=False),
            'Unternehmen': df['Security'].str.strip(),
            'Sektor': df['GICS Sector'].str.strip() if 'GICS Sector' in df.columns else 'K.A.',
            'Branche': df['GICS Sub-Industry'].str.strip() if 'GICS Sub-Industry' in df.columns else 'K.A.'
        })
        
        print(f"Erfolgreich {len(result)} S&P 500 Ticker geladen!")
        return result
        
    except Exception as e:
        print(f"Fehler: {e}")
        print("Verwende Fallback-Tickerliste...")
        fallback = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'BRK-B', 'UNH', 'JNJ']
        return pd.DataFrame({'Symbol': fallback, 'Unternehmen': fallback, 
                            'Sektor': ['Diverse']*len(fallback), 'Branche': ['Diverse']*len(fallback)})

def hole_sp500_index_daten(start_datum, end_datum):
    """
    S&P 500 Indexdaten für Relative-Stärke-Berechnung abrufen.
    """
    try:
        spx = yf.Ticker('^GSPC')
        hist = spx.history(start=start_datum, end=end_datum, auto_adjust=True)
        return hist['Close'] if not hist.empty else None
    except:
        return None

# Ticker abrufen
sp500_df = hole_sp500_ticker()
print(f"\nErste 5 Ticker: {', '.join(sp500_df['Symbol'].head(5).tolist())}")

## 4. RSL-Berechnungs-Engine

In [ ]:
def berechne_rsl(kurse, periode):
    """RSL für eine Kursreihe berechnen."""
    if kurse is None or len(kurse) < periode:
        return None
    try:
        aktueller_kurs = kurse.iloc[-1]
        sma = kurse.iloc[-periode:].mean()
        if sma == 0 or pd.isna(sma):
            return None
        return round(aktueller_kurs / sma, 4)
    except:
        return None

def berechne_aenderung(kurse, tage):
    """Prozentuale Kursänderung über N Tage berechnen."""
    if kurse is None or len(kurse) < tage:
        return None
    try:
        aktuell = kurse.iloc[-1]
        vorher = kurse.iloc[-tage]
        if vorher == 0:
            return None
        return round(((aktuell - vorher) / vorher) * 100, 2)
    except:
        return None

def berechne_gleitender_durchschnitt(kurse, periode):
    """Einfachen gleitenden Durchschnitt berechnen."""
    if kurse is None or len(kurse) < periode:
        return None
    try:
        return round(kurse.iloc[-periode:].mean(), 2)
    except:
        return None

def hole_aktien_daten(ticker, start_datum, end_datum, spx_kurse=None):
    """
    Umfassende Aktiendaten von Yahoo Finance abrufen.
    """
    try:
        aktie = yf.Ticker(ticker)
        
        # Historische Kurse
        hist = aktie.history(start=start_datum, end=end_datum, auto_adjust=True)
        if hist.empty or len(hist) < RSL_PERIODE_6M:
            return None
        
        schlusskurse = hist['Close']
        volumen = hist['Volume']
        
        # Fundamentaldaten
        info = aktie.info
        
        # Aktuelle Werte
        aktueller_kurs = schlusskurse.iloc[-1]
        kurs_52w_hoch = schlusskurse.max()
        kurs_52w_tief = schlusskurse.min()
        
        # RSL für verschiedene Zeiträume
        rsl_3m = berechne_rsl(schlusskurse, RSL_PERIODE_3M)
        rsl_6m = berechne_rsl(schlusskurse, RSL_PERIODE_6M)
        rsl_12m = berechne_rsl(schlusskurse, RSL_PERIODE_12M)
        
        # Kursänderungen
        aenderung_1m = berechne_aenderung(schlusskurse, 21)
        aenderung_3m = berechne_aenderung(schlusskurse, 63)
        aenderung_6m = berechne_aenderung(schlusskurse, 130)
        aenderung_12m = berechne_aenderung(schlusskurse, 252)
        
        # Gleitende Durchschnitte
        ma_50 = berechne_gleitender_durchschnitt(schlusskurse, MA_50)
        ma_200 = berechne_gleitender_durchschnitt(schlusskurse, MA_200)
        
        # Relative Stärke vs S&P 500
        rel_staerke_spx = None
        if spx_kurse is not None and len(spx_kurse) >= RSL_PERIODE_6M:
            aktie_perf = berechne_aenderung(schlusskurse, RSL_PERIODE_6M)
            spx_perf = berechne_aenderung(spx_kurse, RSL_PERIODE_6M)
            if aktie_perf is not None and spx_perf is not None:
                rel_staerke_spx = round(aktie_perf - spx_perf, 2)
        
        # Tage seit 52-Wochen-Hoch
        try:
            hoch_idx = schlusskurse.idxmax()
            tage_seit_hoch = (schlusskurse.index[-1] - hoch_idx).days
        except:
            tage_seit_hoch = None
        
        # Durchschnittsvolumen
        durchschn_volumen = volumen.iloc[-20:].mean() if len(volumen) >= 20 else volumen.mean()
        
        return {
            'Aktueller_Kurs': round(aktueller_kurs, 2),
            'Marktkapitalisierung': info.get('marketCap', None),
            'RSL_3M': rsl_3m,
            'RSL_6M': rsl_6m,
            'RSL_12M': rsl_12m,
            'Aenderung_1M': aenderung_1m,
            'Aenderung_3M': aenderung_3m,
            'Aenderung_6M': aenderung_6m,
            'Aenderung_12M': aenderung_12m,
            'MA_50': ma_50,
            'MA_200': ma_200,
            '52W_Hoch': round(kurs_52w_hoch, 2),
            '52W_Tief': round(kurs_52w_tief, 2),
            'Proz_vom_Hoch': round(((aktueller_kurs - kurs_52w_hoch) / kurs_52w_hoch) * 100, 2),
            'Proz_vom_Tief': round(((aktueller_kurs - kurs_52w_tief) / kurs_52w_tief) * 100, 2),
            'Tage_seit_Hoch': tage_seit_hoch,
            'Durchschn_Volumen': int(durchschn_volumen) if durchschn_volumen else None,
            'Beta': info.get('beta', None),
            'KGV': info.get('trailingPE', None),
            'Dividendenrendite': round(info.get('dividendYield', 0) * 100, 2) if info.get('dividendYield') else 0,
            'Rel_Staerke_SPX': rel_staerke_spx,
            'Datenpunkte': len(schlusskurse)
        }
        
    except Exception as e:
        return None

print("RSL-Berechnungsfunktionen definiert!")

## 5. Datenverarbeitung

In [ ]:
def verarbeite_alle_aktien(ticker_df, start_datum, end_datum, spx_kurse):
    """
    Alle Aktien verarbeiten mit Fortschrittsanzeige.
    """
    ergebnisse = []
    fehlgeschlagen = []
    
    print(f"\nVerarbeite {len(ticker_df)} Aktien...")
    print(f"Zeitraum: {start_datum.strftime('%d.%m.%Y')} bis {end_datum.strftime('%d.%m.%Y')}")
    print("Dies kann 10-15 Minuten dauern...\n")
    
    for idx, zeile in tqdm(ticker_df.iterrows(), total=len(ticker_df), desc="Lade Daten"):
        ticker = zeile['Symbol']
        
        daten = hole_aktien_daten(ticker, start_datum, end_datum, spx_kurse)
        
        if daten and daten.get('RSL_6M') is not None:
            ergebnisse.append({
                'Ticker': ticker,
                'Unternehmen': zeile['Unternehmen'],
                'Sektor': zeile['Sektor'],
                'Branche': zeile['Branche'],
                **daten
            })
        else:
            fehlgeschlagen.append(ticker)
        
        time.sleep(API_VERZOEGERUNG)
    
    print(f"\n{'='*60}")
    print(f"Verarbeitung abgeschlossen!")
    print(f"  - Erfolgreich: {len(ergebnisse)} Aktien")
    print(f"  - Fehlgeschlagen: {len(fehlgeschlagen)} Aktien")
    
    # DataFrame erstellen und sortieren
    df = pd.DataFrame(ergebnisse)
    df = df.sort_values('RSL_6M', ascending=False).reset_index(drop=True)
    df.insert(0, 'Rang', range(1, len(df) + 1))
    
    # Perzentil berechnen
    df['Perzentil'] = df['RSL_6M'].rank(pct=True).apply(lambda x: round(x * 100, 1))
    
    return df, fehlgeschlagen

# Datumsbereich
end_datum = datetime.now()
start_datum = end_datum - timedelta(days=RUECKBLICK_TAGE)

# S&P 500 Indexdaten laden
print("Lade S&P 500 Index-Referenzdaten...")
spx_kurse = hole_sp500_index_daten(start_datum, end_datum)
if spx_kurse is not None:
    print(f"S&P 500 Daten geladen: {len(spx_kurse)} Datenpunkte")
else:
    print("Warnung: S&P 500 Daten konnten nicht geladen werden")

# Alle Aktien verarbeiten
ergebnis_df, fehlgeschlagene_ticker = verarbeite_alle_aktien(sp500_df, start_datum, end_datum, spx_kurse)

## 6. Analyse & Statistiken

In [ ]:
# Zusammenfassungsstatistiken
print("=" * 70)
print("RSL ANALYSE - ZUSAMMENFASSUNG")
print("=" * 70)
print(f"\nAnalysierte Aktien: {len(ergebnis_df)}")
print(f"\nRSL-6M Statistiken:")
print(f"  - Durchschnitt: {ergebnis_df['RSL_6M'].mean():.4f}")
print(f"  - Median:       {ergebnis_df['RSL_6M'].median():.4f}")
print(f"  - Maximum:      {ergebnis_df['RSL_6M'].max():.4f} ({ergebnis_df.loc[ergebnis_df['RSL_6M'].idxmax(), 'Ticker']})")
print(f"  - Minimum:      {ergebnis_df['RSL_6M'].min():.4f} ({ergebnis_df.loc[ergebnis_df['RSL_6M'].idxmin(), 'Ticker']})")

# Momentum-Verteilung
bullisch = (ergebnis_df['RSL_6M'] > 1.0).sum()
baerisch = (ergebnis_df['RSL_6M'] < 1.0).sum()
print(f"\nMomentum-Verteilung:")
print(f"  - Bullisch (RSL > 1,0): {bullisch} ({bullisch/len(ergebnis_df)*100:.1f}%)")
print(f"  - Bärisch (RSL < 1,0):  {baerisch} ({baerisch/len(ergebnis_df)*100:.1f}%)")

In [ ]:
# Top 10 anzeigen
print("\n" + "=" * 70)
print("TOP 10 AKTIEN NACH RSL-6M")
print("=" * 70 + "\n")

top10 = ergebnis_df.head(10)[['Rang', 'Ticker', 'Unternehmen', 'Sektor', 'RSL_6M', 
                              'Aenderung_6M', 'Aktueller_Kurs', 'Rel_Staerke_SPX']]
print(top10.to_string(index=False))

In [ ]:
# Top 25% Portfolio
top_schwelle = int(len(ergebnis_df) * TOP_PROZENT)
top_performer = ergebnis_df.head(top_schwelle).copy()

print("\n" + "=" * 70)
print(f"OBERE 25% PORTFOLIO ({len(top_performer)} Aktien)")
print("=" * 70)
print(f"\nRSL-Schwellenwert: >= {top_performer['RSL_6M'].min():.4f}")
print(f"Durchschnitt RSL: {top_performer['RSL_6M'].mean():.4f}")
print(f"Durchschnitt 6M-Rendite: {top_performer['Aenderung_6M'].mean():.2f}%")

print(f"\nSektorverteilung in Top 25%:")
sektor_top = top_performer['Sektor'].value_counts()
for sektor, anzahl in sektor_top.head(5).items():
    print(f"  - {sektor}: {anzahl} ({anzahl/len(top_performer)*100:.1f}%)")

In [ ]:
# Sektoranalyse
print("\n" + "=" * 70)
print("SEKTORANALYSE")
print("=" * 70 + "\n")

sektor_stats = ergebnis_df.groupby('Sektor').agg({
    'RSL_6M': ['mean', 'median', 'count'],
    'Aenderung_6M': 'mean'
}).round(4)
sektor_stats.columns = ['Durchschn_RSL', 'Median_RSL', 'Anzahl', 'Durchschn_6M_Aend']
sektor_stats = sektor_stats.sort_values('Durchschn_RSL', ascending=False)

# Anzahl in Top 25% pro Sektor
sektor_top25 = top_performer.groupby('Sektor').size().reindex(sektor_stats.index, fill_value=0)
sektor_stats['In_Top_25%'] = sektor_top25
sektor_stats['Anteil_Top25'] = (sektor_stats['In_Top_25%'] / sektor_stats['Anzahl'] * 100).round(1)

print(sektor_stats.to_string())

## 7. Excel-Bericht erstellen

In [ ]:
def formatiere_marktkapitalisierung(wert):
    """Marktkapitalisierung in lesbares Format umwandeln."""
    if wert is None or pd.isna(wert):
        return 'K.A.'
    if wert >= 1e12:
        return f"{wert/1e12:.2f} Bio."
    elif wert >= 1e9:
        return f"{wert/1e9:.2f} Mrd."
    elif wert >= 1e6:
        return f"{wert/1e6:.2f} Mio."
    else:
        return f"{wert:,.0f}"

def erstelle_excel_bericht(ergebnis_df, top_performer, sektor_stats, fehlgeschlagene, ausgabe_datei):
    """
    Erstellt professionellen Excel-Bericht mit 5 Blättern.
    """
    print(f"\nErstelle Excel-Bericht: {ausgabe_datei}")
    
    # Kopie für Excel-Formatierung
    excel_df = ergebnis_df.copy()
    excel_df['Marktkapitalisierung_Text'] = excel_df['Marktkapitalisierung'].apply(formatiere_marktkapitalisierung)
    
    with pd.ExcelWriter(ausgabe_datei, engine='xlsxwriter') as writer:
        workbook = writer.book
        
        # Formate definieren
        header_format = workbook.add_format({
            'bold': True, 'bg_color': '#1F4E79', 'font_color': 'white',
            'border': 1, 'align': 'center', 'valign': 'vcenter', 'text_wrap': True
        })
        
        zahl_format = workbook.add_format({'num_format': '#,##0.00', 'border': 1})
        proz_format = workbook.add_format({'num_format': '0.00%', 'border': 1})
        waehrung_format = workbook.add_format({'num_format': '$#,##0.00', 'border': 1})
        
        gruen_format = workbook.add_format({'bg_color': '#C6EFCE', 'border': 1})
        rot_format = workbook.add_format({'bg_color': '#FFC7CE', 'border': 1})
        gelb_format = workbook.add_format({'bg_color': '#FFEB9C', 'border': 1})
        
        titel_format = workbook.add_format({'bold': True, 'font_size': 14, 'font_color': '#1F4E79'})
        
        # =====================================================================
        # BLATT 1: Vollständige Rangliste
        # =====================================================================
        blatt1_spalten = ['Rang', 'Ticker', 'Unternehmen', 'Sektor', 'Branche', 'Aktueller_Kurs',
                         'Marktkapitalisierung_Text', 'RSL_3M', 'RSL_6M', 'RSL_12M', 'Perzentil',
                         'Aenderung_1M', 'Aenderung_3M', 'Aenderung_6M', 'Aenderung_12M',
                         'MA_50', 'MA_200', 'Proz_vom_Hoch', 'Proz_vom_Tief',
                         'Durchschn_Volumen', 'Beta', 'KGV', 'Dividendenrendite']
        
        blatt1_df = excel_df[blatt1_spalten].copy()
        blatt1_df.columns = ['Rang', 'Ticker', 'Unternehmen', 'Sektor', 'Branche', 'Kurs ($)',
                            'Marktkapitalisierung', 'RSL 3M', 'RSL 6M', 'RSL 12M', 'Perzentil',
                            'Änd. 1M (%)', 'Änd. 3M (%)', 'Änd. 6M (%)', 'Änd. 12M (%)',
                            'MA 50', 'MA 200', '% vom Hoch', '% vom Tief',
                            'Ø Volumen', 'Beta', 'KGV', 'Div. Rendite (%)']
        
        blatt1_df.to_excel(writer, sheet_name='Vollstaendige_Rangliste', index=False, startrow=0)
        ws1 = writer.sheets['Vollstaendige_Rangliste']
        
        # Header formatieren
        for col_num, value in enumerate(blatt1_df.columns):
            ws1.write(0, col_num, value, header_format)
        
        # Spaltenbreiten
        ws1.set_column('A:A', 6)
        ws1.set_column('B:B', 8)
        ws1.set_column('C:C', 25)
        ws1.set_column('D:E', 18)
        ws1.set_column('F:W', 12)
        
        # Bedingte Formatierung für Top 25%
        top25_schwelle = len(ergebnis_df) * 0.25
        ws1.conditional_format(1, 0, int(top25_schwelle), len(blatt1_df.columns)-1, {
            'type': 'formula',
            'criteria': '=$A2<=' + str(int(top25_schwelle)),
            'format': gruen_format
        })
        
        ws1.freeze_panes(1, 0)
        ws1.autofilter(0, 0, len(blatt1_df), len(blatt1_df.columns)-1)
        
        # =====================================================================
        # BLATT 2: Top 25% Stars
        # =====================================================================
        top_excel = top_performer.copy()
        top_excel['Marktkapitalisierung_Text'] = top_excel['Marktkapitalisierung'].apply(formatiere_marktkapitalisierung)
        
        blatt2_spalten = ['Rang', 'Ticker', 'Unternehmen', 'Sektor', 'Aktueller_Kurs',
                         'Marktkapitalisierung_Text', 'RSL_6M', 'Perzentil', 'Rel_Staerke_SPX',
                         'Aenderung_6M', 'Tage_seit_Hoch', 'Proz_vom_Hoch', 'KGV', 'Dividendenrendite']
        
        blatt2_df = top_excel[blatt2_spalten].copy()
        blatt2_df.columns = ['Rang', 'Ticker', 'Unternehmen', 'Sektor', 'Kurs ($)',
                            'Marktkapitalisierung', 'RSL 6M', 'Perzentil', 'Rel. Stärke vs SPX',
                            'Änd. 6M (%)', 'Tage seit Hoch', '% vom Hoch', 'KGV', 'Div. Rendite (%)']
        
        blatt2_df.to_excel(writer, sheet_name='Top_25%_Stars', index=False, startrow=0)
        ws2 = writer.sheets['Top_25%_Stars']
        
        for col_num, value in enumerate(blatt2_df.columns):
            ws2.write(0, col_num, value, header_format)
        
        ws2.set_column('A:A', 6)
        ws2.set_column('B:B', 8)
        ws2.set_column('C:C', 25)
        ws2.set_column('D:N', 14)
        ws2.freeze_panes(1, 0)
        
        # =====================================================================
        # BLATT 3: Sektoranalyse
        # =====================================================================
        # Beste Aktie pro Sektor finden
        beste_pro_sektor = ergebnis_df.loc[ergebnis_df.groupby('Sektor')['RSL_6M'].idxmax()]
        beste_dict = beste_pro_sektor.set_index('Sektor')['Ticker'].to_dict()
        
        sektor_df = sektor_stats.reset_index()
        sektor_df['Beste_Aktie'] = sektor_df['Sektor'].map(beste_dict)
        sektor_df.columns = ['Sektor', 'Ø RSL', 'Median RSL', 'Anzahl Aktien', 'Ø 6M Änd. (%)',
                            'In Top 25%', 'Anteil Top 25% (%)', 'Beste Aktie']
        
        sektor_df.to_excel(writer, sheet_name='Sektoranalyse', index=False, startrow=0)
        ws3 = writer.sheets['Sektoranalyse']
        
        for col_num, value in enumerate(sektor_df.columns):
            ws3.write(0, col_num, value, header_format)
        
        ws3.set_column('A:A', 25)
        ws3.set_column('B:H', 15)
        ws3.freeze_panes(1, 0)
        
        # =====================================================================
        # BLATT 4: Zusammenfassung
        # =====================================================================
        ws4 = workbook.add_worksheet('Zusammenfassung')
        
        ws4.write(0, 0, 'RSL SCREENING - ZUSAMMENFASSUNG', titel_format)
        ws4.write(2, 0, 'Analysedatum:', header_format)
        ws4.write(2, 1, datetime.now().strftime('%d.%m.%Y %H:%M'))
        ws4.write(3, 0, 'Analysierte Aktien:', header_format)
        ws4.write(3, 1, len(ergebnis_df))
        ws4.write(4, 0, 'Fehlgeschlagene Ticker:', header_format)
        ws4.write(4, 1, len(fehlgeschlagene))
        
        ws4.write(6, 0, 'RSL-6M STATISTIKEN', titel_format)
        stats = [
            ('Durchschnitt', f"{ergebnis_df['RSL_6M'].mean():.4f}"),
            ('Median', f"{ergebnis_df['RSL_6M'].median():.4f}"),
            ('Maximum', f"{ergebnis_df['RSL_6M'].max():.4f}"),
            ('Minimum', f"{ergebnis_df['RSL_6M'].min():.4f}"),
            ('Standardabweichung', f"{ergebnis_df['RSL_6M'].std():.4f}"),
        ]
        for i, (name, wert) in enumerate(stats):
            ws4.write(8+i, 0, name, header_format)
            ws4.write(8+i, 1, wert)
        
        # Top 10 Schnellreferenz
        ws4.write(15, 0, 'TOP 10 AKTIEN', titel_format)
        ws4.write(17, 0, 'Rang', header_format)
        ws4.write(17, 1, 'Ticker', header_format)
        ws4.write(17, 2, 'RSL 6M', header_format)
        ws4.write(17, 3, 'Änd. 6M (%)', header_format)
        
        for i, (_, row) in enumerate(ergebnis_df.head(10).iterrows()):
            ws4.write(18+i, 0, row['Rang'])
            ws4.write(18+i, 1, row['Ticker'])
            ws4.write(18+i, 2, row['RSL_6M'])
            ws4.write(18+i, 3, row['Aenderung_6M'])
        
        # Bottom 10
        ws4.write(15, 5, 'BOTTOM 10 AKTIEN', titel_format)
        ws4.write(17, 5, 'Rang', header_format)
        ws4.write(17, 6, 'Ticker', header_format)
        ws4.write(17, 7, 'RSL 6M', header_format)
        ws4.write(17, 8, 'Änd. 6M (%)', header_format)
        
        for i, (_, row) in enumerate(ergebnis_df.tail(10).iterrows()):
            ws4.write(18+i, 5, row['Rang'])
            ws4.write(18+i, 6, row['Ticker'])
            ws4.write(18+i, 7, row['RSL_6M'])
            ws4.write(18+i, 8, row['Aenderung_6M'])
        
        ws4.set_column('A:A', 20)
        ws4.set_column('B:I', 12)
        
        # =====================================================================
        # BLATT 5: Methodik
        # =====================================================================
        ws5 = workbook.add_worksheet('Methodik')
        
        methodik_text = [
            ('RSL SCREENING - METHODIK', True),
            ('', False),
            ('1. RSL-BERECHNUNG (Relative Stärke Levy)', True),
            ('Die RSL wird berechnet als: Aktueller Kurs / SMA(Kurs, N Perioden)', False),
            ('Verwendete Perioden: 3 Monate (63 Tage), 6 Monate (130 Tage), 12 Monate (252 Tage)', False),
            ('', False),
            ('2. INTERPRETATION', True),
            ('RSL > 1,0: Aktie handelt über ihrem Durchschnitt (bullisches Momentum)', False),
            ('RSL < 1,0: Aktie handelt unter ihrem Durchschnitt (bärisches Momentum)', False),
            ('Je höher der RSL-Wert, desto stärker das relative Momentum', False),
            ('', False),
            ('3. DATENQUELLEN', True),
            ('Aktienticker: Wikipedia (Liste der S&P 500 Unternehmen)', False),
            ('Kursdaten: Yahoo Finance (yfinance API)', False),
            ('Referenzindex: S&P 500 (^GSPC)', False),
            ('', False),
            ('4. TOP 25% AUSWAHL', True),
            ('Aktien werden nach RSL-6M absteigend sortiert', False),
            ('Die oberen 25% werden als "Stars" / Kaufkandidaten hervorgehoben', False),
            ('', False),
            ('5. HAFTUNGSAUSSCHLUSS', True),
            ('Dieses Tool dient ausschließlich Bildungs- und Forschungszwecken.', False),
            ('Vergangene Performance garantiert keine zukünftigen Ergebnisse.', False),
            ('Konsultieren Sie einen Finanzberater vor Investitionsentscheidungen.', False),
            ('', False),
            (f'Letzte Aktualisierung: {datetime.now().strftime("%d.%m.%Y %H:%M:%S")}', False),
        ]
        
        for i, (text, is_header) in enumerate(methodik_text):
            if is_header and text:
                ws5.write(i, 0, text, titel_format)
            else:
                ws5.write(i, 0, text)
        
        ws5.set_column('A:A', 80)
    
    print(f"\nExcel-Bericht erfolgreich erstellt!")
    print(f"Datei: {ausgabe_datei}")
    return ausgabe_datei

# Excel-Bericht erstellen
excel_datei = erstelle_excel_bericht(ergebnis_df, top_performer, sektor_stats, fehlgeschlagene_ticker, AUSGABE_DATEI)

## 8. Download (Google Colab)

In [ ]:
# Datei in Google Colab herunterladen
try:
    from google.colab import files
    files.download(excel_datei)
    print(f"\nDownload gestartet: {excel_datei}")
except ImportError:
    print(f"\nNicht in Google Colab.")
    print(f"Excel-Datei gespeichert: {os.path.abspath(excel_datei)}")
except Exception as e:
    print(f"Download-Fehler: {e}")
    print(f"Datei unter: {os.path.abspath(excel_datei)}")

## 9. Abschlusszusammenfassung

In [ ]:
print("\n" + "=" * 70)
print("RSL SCREENING ABGESCHLOSSEN")
print("=" * 70)
print(f"\nBericht: {AUSGABE_DATEI}")
print(f"Zeitpunkt: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")
print(f"\nErgebnisse:")
print(f"  - Analysierte Aktien: {len(ergebnis_df)}")
print(f"  - Top-Performer: {ergebnis_df.iloc[0]['Ticker']} (RSL = {ergebnis_df.iloc[0]['RSL_6M']:.4f})")
print(f"  - Schwächste Aktie: {ergebnis_df.iloc[-1]['Ticker']} (RSL = {ergebnis_df.iloc[-1]['RSL_6M']:.4f})")
print(f"  - Obere 25% Kaufkandidaten: {len(top_performer)} Aktien")
print(f"\nExcel-Blätter:")
print(f"  1. Vollstaendige_Rangliste - Alle {len(ergebnis_df)} Aktien")
print(f"  2. Top_25%_Stars - {len(top_performer)} Kaufkandidaten")
print(f"  3. Sektoranalyse - Branchenvergleich")
print(f"  4. Zusammenfassung - Statistiken & Top/Bottom 10")
print(f"  5. Methodik - Erklärung & Haftungsausschluss")
print("\n" + "=" * 70)

---

## Hinweise zur Nutzung

### Strategie-Empfehlungen
1. **Kaufkandidaten**: Obere 25% nach RSL-6M
2. **Vermeiden**: Untere 25% (schwaches Momentum)
3. **Rebalancing**: Monatlich oder vierteljährlich
4. **Diversifikation**: Über mehrere Sektoren streuen

### Wichtige Kennzahlen
- **RSL > 1,05**: Starkes bullisches Momentum
- **RSL 1,00-1,05**: Leicht bullisch
- **RSL 0,95-1,00**: Leicht bärisch
- **RSL < 0,95**: Starkes bärisches Momentum

### Haftungsausschluss
Dieses Tool dient ausschließlich Bildungs- und Forschungszwecken. Vergangene Performance garantiert keine zukünftigen Ergebnisse.

---
*Erstellt mit Python | Daten: Yahoo Finance & Wikipedia*